In [ ]:
import requests
import csv
import threading
from queue import Queue
import calendar

API_KEY = "api_key"  # Reemplaza con tu clave de TMDb
BASE_URL = "https://api.themoviedb.org/3/discover/movie"
NUM_THREADS = 25  # Número de threads en paralelo


# Archivo CSV para exportar los datos
csv_filename = "movies_tmdb_filt.csv"
lock = threading.Lock()

# Lista para almacenar los datos temporalmente
movies_data = []

def fetch_data(year, month, day, page_queue):
    while not page_queue.empty():
        page = page_queue.get()
        params = {
            "api_key": API_KEY,
            "language": "es-ES",
            "sort_by": "release_date.asc",
            "primary_release_year": year,
            "watch_region": "AR",
            "with_watch_providers": "337|8|119|2|3|350|531|339|167|1899|2302",
            "page": page
        }
        if day:
            params["primary_release_date.gte"] = f"{year}-{month:02d}-{day:02d}"
            params["primary_release_date.lte"] = f"{year}-{month:02d}-{day:02d}"
        elif month:
            params["primary_release_date.gte"] = f"{year}-{month:02d}-01"
            params["primary_release_date.lte"] = f"{year}-{month:02d}-{calendar.monthrange(year, month)[1]}"
        
        response = requests.get(BASE_URL, params=params)
        
        if response.status_code == 200:
            results = response.json().get("results", [])
            page_data = [(
                movie.get('id'),
                movie.get('title', 'Desconocido'),
                movie.get('release_date', 'Desconocido'),
                movie.get('backdrop_path', 'N/A'),
                movie.get('genre_ids', []),
                movie.get('original_language', 'N/A'),
                movie.get('original_title', 'Desconocido'),
                movie.get('overview', 'N/A'),
                movie.get('popularity', 0),
                movie.get('poster_path', 'N/A'),
                movie.get('vote_average', 0),
                movie.get('vote_count', 0)
            ) for movie in results]
            
            with lock:
                movies_data.extend(page_data)
        else:
            print(f"⚠️ Error en la página {page} del año {year}, mes {month}, día {day}: {response.status_code} - {response.text}")
        
        page_queue.task_done()

# Iterar por cada año desde 1930 hasta 2025
for year in range(1890, 2025):
    print(f"📅 Procesando el año: {year}")
    page_queue = Queue()
    
    params = {
        "api_key": API_KEY,
        "language": "es-ES",
        "sort_by": "release_date.asc",
        "primary_release_year": year,
        "watch_region": "AR",
        "with_watch_providers": "337|8|119|2|3|350|531|339|167|1899|2302",
        "page": 1
    }
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        total_pages = response.json().get("total_pages", 1)
        
        if total_pages > 500:
            print(f"⚠️ El año {year} tiene más de 500 páginas, iterando por meses.")
            for month in range(1, 13):
                print(f"📆 Procesando el mes: {month} de {year}")
                page_queue = Queue()
                params["primary_release_date.gte"] = f"{year}-{month:02d}-01"
                params["primary_release_date.lte"] = f"{year}-{month:02d}-{calendar.monthrange(year, month)[1]}"
                response = requests.get(BASE_URL, params=params)
                
                if response.status_code == 200:
                    month_pages = response.json().get("total_pages", 1)
                    print(f"📄 Total de páginas para {year}-{month}: {month_pages}")
                    
                    if month_pages > 500:
                        print(f"⚠️ El mes {month} de {year} tiene más de 500 páginas, iterando por días.")
                        for day in range(1, calendar.monthrange(year, month)[1] + 1):
                            page_queue = Queue()
                            params["primary_release_date.gte"] = f"{year}-{month:02d}-{day:02d}"
                            params["primary_release_date.lte"] = f"{year}-{month:02d}-{day:02d}"
                            response = requests.get(BASE_URL, params=params)
                            
                            if response.status_code == 200:
                                day_pages = response.json().get("total_pages", 1)
                                print(f"📄 Total de páginas para {year}-{month}-{day}: {day_pages}")
                                for page in range(1, min(day_pages, 500) + 1):
                                    page_queue.put(page)
                                
                                threads = []
                                for _ in range(NUM_THREADS):
                                    thread = threading.Thread(target=fetch_data, args=(year, month, day, page_queue))
                                    thread.start()
                                    threads.append(thread)
                                
                                for thread in threads:
                                    thread.join()
                    else:
                        for page in range(1, min(month_pages, 500) + 1):
                            page_queue.put(page)
                        
                        threads = []
                        for _ in range(NUM_THREADS):
                            thread = threading.Thread(target=fetch_data, args=(year, month, None, page_queue))
                            thread.start()
                            threads.append(thread)
                        
                        for thread in threads:
                            thread.join()
        else:
            print(f"📢 Total de páginas para {year}: {total_pages}")
            for page in range(1, total_pages + 1):
                page_queue.put(page)
            
            threads = []
            for _ in range(NUM_THREADS):
                thread = threading.Thread(target=fetch_data, args=(year, None, None, page_queue))
                thread.start()
                threads.append(thread)
            
            for thread in threads:
                thread.join()
    else:
        print(f"⚠️ Error al obtener datos iniciales para {year}: {response.status_code} - {response.text}")

# Guardar los datos en un archivo CSV
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["ID", "Título", "Fecha de Estreno", "Backdrop Path", "Géneros", "Idioma Original", "Título Original", "Descripción", "Popularidad", "Poster Path", "Puntaje Promedio", "Cantidad de Votos"])
    writer.writerows(movies_data)

print(f"✅ Archivo CSV generado: {csv_filename}")


📅 Procesando el año: 1890
📢 Total de páginas para 1890: 1
📅 Procesando el año: 1891
📢 Total de páginas para 1891: 1
📅 Procesando el año: 1892
📢 Total de páginas para 1892: 1
📅 Procesando el año: 1893
📢 Total de páginas para 1893: 1
📅 Procesando el año: 1894
📢 Total de páginas para 1894: 1
📅 Procesando el año: 1895
📢 Total de páginas para 1895: 1
📅 Procesando el año: 1896
📢 Total de páginas para 1896: 1
📅 Procesando el año: 1897
📢 Total de páginas para 1897: 1
📅 Procesando el año: 1898
📢 Total de páginas para 1898: 1
📅 Procesando el año: 1899
📢 Total de páginas para 1899: 1
📅 Procesando el año: 1900
📢 Total de páginas para 1900: 1
📅 Procesando el año: 1901
📢 Total de páginas para 1901: 1
📅 Procesando el año: 1902
📢 Total de páginas para 1902: 1
📅 Procesando el año: 1903
📢 Total de páginas para 1903: 1
📅 Procesando el año: 1904
📢 Total de páginas para 1904: 1
📅 Procesando el año: 1905
📢 Total de páginas para 1905: 1
📅 Procesando el año: 1906
📢 Total de páginas para 1906: 1
📅 Procesando e